# Modulok, Adatok beolvasása

In [ ]:
import zipfile
import pandas as pd
import os
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
dir = '/temp/mydata'

In [ ]:
if not os.path.exists(dir):
    os.mkdir(dir)
os.chdir(dir)
for f in os.listdir():
  os.remove(f)
!wget --quiet https://github.com/bepti/colab/raw/main/data.zip 

In [ ]:
os.chdir(dir) #ha esetleg az előző cella kimaradna... 
with zipfile.ZipFile('data.zip') as myzip:
  myzip.extractall()
files = [file for file in os.listdir() if file[-4:]=='.dat'] #csak a .dat fájlok
files.sort()

In [ ]:
# fájlok beolvasása és dataframe generálása
# a mikrokontroller adottságai miatt bináris szervezésű fájlok
rows = []
for dfile in files:
  f = open(dfile,'rb')
  exec=dfile[:-14]
  start=dfile[-14:-4]
  b = f.read(16)
  while len(b)>0:
    time_ms = int.from_bytes(b[0:4], byteorder='big', signed=True)
    imus = [exec,start,time_ms]
    for i in range(6):
      imus.append(int.from_bytes(b[i*2+4:i*2+6], byteorder='big', signed=True))
    rows.append(imus)
    b = f.read(16)  
  f.close()
print('number of samples:',len(rows))
df = pd.DataFrame(rows,columns=['exec','start','time_ms','acc_x','acc_y','acc_z','gyr_x','gyr_y','gyr_z'])
df.head()

# Feature felderítés transzformáció vizualizálás

In [ ]:
# >
#max szórású csatornák
samples = pd.DataFrame()
fig, axs = plt.subplots(2, 5, figsize=(30,10))
samples=df.groupby(['exec','start'], as_index=False)['exec'].count()
for i in range(0,10):
    start = samples.iloc[i,0]
    dff = df[df['start']==start].copy().reset_index(drop=True)
    max_dev_var = dff.describe().loc['std','acc_x':].idxmax()
    ax=axs[i%10//5,i%5]
    dff[max_dev_var].plot(ax=ax, title=dff.loc[0,'exec'] +' - '+ dff.loc[0,'start'], label=max_dev_var)
    #ax.vlines([50,ax.get_xlim()[1]-100],ax.get_ylim()[0],ax.get_ylim()[1], colors=['g','r'])
    ax.hlines([0],ax.get_xlim()[0],ax.get_xlim()[1], colors=['grey'])
    ax.legend()

In [ ]:
# >
# Megpróbálom periódusokra vágni a mintákat majd öszefűzni a gyakorlatokat:
# Kiválasztom a hat csatorna közül a legnagyobb szórásút (amplitudó), feltételezve hogy ez a csatorna lesz a legjellemzőbb a periódusokra
# nyilván ennek nem lesz köze a kettlebell gyakorlat "elejéhez" de ez alapjában véve nem lényeges (azt hiszem)
# A szándék, hogy a sok azonos gyakorlatot összefűzöm egy nagy többszáz ciklusos virtuális gyakorlattá és abból walk-forward módon egyforma (átfedő) szeleteket vágok a tanításhoz
start = pd.DataFrame()
for i in range(80,83): #len(samples)):
    start = samples.iloc[i,0]
    dff = df[df['start']==start].copy().reset_index(drop=True)
    max_dev_var = dff.describe().loc['std','acc_x':].idxmax()
    max_dev=dff[max_dev_var].rolling(50, center=True).mean()
    offset=max_dev.quantile(0.28)
    zerocross=(np.sign(max_dev-offset)-np.sign(max_dev.shift(1,fill_value=0)-offset))/2
    plt.figure(figsize=(20,7))
    plt.plot(dff.index,max_dev)
    plt.plot(dff.index,dff[max_dev_var])
    plt.hlines([offset],dff.index.min(),dff.index.max() )
    plt.vlines(zerocross[zerocross==1].index,offset-1000,offset+1000)
    plt.vlines([zerocross[zerocross==1].index[0],zerocross[zerocross==1].index[-1]],offset-4000,offset+4000,colors=['r'])
    plt.show()

In [ ]:
# egy gyakorlat - összes csatorna interaktív vizualizációja
dff = df[df['start']==start].copy().reset_index(drop=True)
fig = go.Figure(layout_title_text=dff.iloc[0]['start']+' '+dff.iloc[0]['exec'])
fig.add_trace(go.Scatter(x=dff.index,y=df['acc_x'],
                  mode='lines', name='gyorsulX',
                hovertext=df['time_ms']))
fig.add_trace(go.Scatter(x=dff.index,y=df['acc_y'],
                  mode='lines', name='gyorsulY', hovertext=df['time_ms']))
fig.add_trace(go.Scatter(x=dff.index,y=df['acc_z'],
                  mode='lines', name='gyorsulZ', hovertext=df['time_ms']))
fig.add_trace(go.Scatter(x=dff.index,y=df['gyr_x'],
                  mode='lines', name='fordulX', hovertext=df['time_ms']))
fig.add_trace(go.Scatter(x=dff.index,y=df['gyr_y'],
                  mode='lines', name='fordulY', hovertext=df['time_ms']))
fig.add_trace(go.Scatter(x=dff.index,y=df['gyr_z'],
                  mode='lines', name='fordulZ', hovertext=df['time_ms']))
fig.update_layout(hovermode="x")
fig.show()

# Adatelőkészítés

In [ ]:
# az azonos gyakorlatokat összekapcsolom úgy hogy a nullaátmeneteknél vágva egy nagy folyamatos 
# gyakorlattá alakuljanak
continous_df = pd.DataFrame(columns=df.columns + ['zerocross'])
for e in np.sort(df['exec'].unique()): #gyakorlat típusokon iterál
  start = np.sort(df['start'][df['exec']==e].unique()) 
  for s in start: #típuson belül gyakorlatokon iterál (fájlok)

    #megragadok egy gyakorlatot rendezem és indexelem időpillanatra    
    dff = df[df['start']==s].copy().sort_values('time_ms').reset_index(drop=True)
    
    #megkeresem a legnagyobb szórást (amplitudót) mutató csatornát
    max_dev_var = dff.describe().loc['std','acc_x':].idxmax() 
    
    #a periódus kereséshez gördülő átlaggal szűröm a mintát (lásd fent a vizualizációt)
    max_dev=dff[max_dev_var].rolling(40, center=True).mean()
    
    #a periódus kereséshez kiválasztom az ideális pszeudo zero tengelyt 
    offset=max_dev.quantile(0.28)
    
    #kijelölöm a mintán a (pszeudo) zero átmenetek helyét 
    zerocross=(np.sign(max_dev-offset)-np.sign(max_dev.shift(1,fill_value=0)-offset))/2 
    
    #kiválasztom az első és utolsó pozitív átmenetet
    period_start = zerocross[zerocross==1].index[0]
    period_end = zerocross[zerocross==1].index[-1]
    dff['zerocross']=zerocross
    
    #a kiválasztott pontokon csonkolt gyakorlatot felfűzöm az egyesített gyakorlatok végére
    continous_df = continous_df.append(dff.loc[period_start:period_end,:])



In [ ]:
# >
# összefűzött gyakorlatok vizualizációja (itt derültek ki félrecimkézések)
for e in ['NOTHING','DEADLIFT','HALO','CLEAN', 'SNATCH','SWING']:
    dff = continous_df[continous_df['exec']==e].copy()
    fig = go.Figure(layout_title_text=dff.iloc[0]['start']+' '+dff.iloc[0]['exec'])
    dff = dff.sort_values(['start','time_ms']).reset_index(drop=True)
    fig.add_trace(go.Scatter(x=dff.index,y=dff['acc_x'],
                      mode='lines', name='gyorsulX', hovertext=dff['start']))
    fig.add_trace(go.Scatter(x=dff.index,y=dff['acc_y'],
                      mode='lines', name='gyorsulY', hovertext=dff['start']))
    fig.add_trace(go.Scatter(x=dff.index,y=dff['acc_z'],
                      mode='lines', name='gyorsulZ', hovertext=dff['time_ms']))
    fig.add_trace(go.Scatter(x=dff.index,y=dff['gyr_x'],
                      mode='lines', name='fordulX', hovertext=dff['exec']))
    fig.add_trace(go.Scatter(x=dff.index,y=dff['gyr_y'],
                      mode='lines', name='fordulY', hovertext=dff['start']))
    fig.add_trace(go.Scatter(x=dff.index,y=dff['gyr_z'],
                      mode='lines', name='fordulZ', hovertext=dff['start']))
    fig.update_layout(hovermode="x")
    fig.show()

In [ ]:
# >
# ismétlések száma az összefűzött mintában gyakorlatonként
for e in ['NOTHING','DEADLIFT','HALO','CLEAN', 'SNATCH','SWING']:
    dff = continous_df[continous_df['exec']==e].copy().reset_index(drop=True)
    print (e, dff['zerocross'][dff['zerocross']==1].count())

In [ ]:
# adatelőkészítés tanításhoz, validáláshoz 
# 200 mintából álló (~5sec) gördülő ablakok kivágása az adatokból
# 30 mintánként csúszik az ablak (750msec)
X = np.array([])
y = np.array([])
j=0
target_dict={0:'DEADLIFT',1:'HALO',2:'CLEAN', 3:'SNATCH',4:'SWING',5:'NOTHING'}
for target, e in target_dict.items():
    dff = continous_df[continous_df['exec']==e].copy().reset_index(drop=True)
    sample_starts = range(0,len(dff)-200,30)   
    for ss in sample_starts:
        if len(dff) > ss+199:
        ################### ezt az alkalmazás során is meg kell csinálni
            X=np.append(X,(dff.loc[ss:ss+199,'acc_x':'gyr_z'].to_numpy()+32768)/65536) 
            y=np.append(y,target)
            j+=1
    X=X.reshape(j,200,6)
print('X:',X.shape)
print('y:',y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from tensorflow.keras.layers import Input, Dense, Dropout, Conv1D, MaxPool1D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adadelta, Adam, SGD
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.backend import clear_session
import tensorflow as tf

In [ ]:
n_classes = 6
dropout_rate_1 = 0.3
dropout_rate_2 = 0.3
batch_size = 25
epoch_count = 50

In [ ]:
tf.compat.v1.reset_default_graph() 
clear_session() 
x = Input(shape=(200,6))
conv1 = Conv1D(16, kernel_size=3 , activation='relu', input_shape=(200,6))(x)
pool1 = MaxPool1D(pool_size=2)(conv1)
dropout1 = Dropout(dropout_rate_1)(pool1)
conv2 = Conv1D(16, kernel_size=3, activation='relu', input_shape=(100,16))(dropout1)
pool2 = MaxPool1D(pool_size=2)(conv2)
dropout2 = Dropout(dropout_rate_1)(pool2)
pool2_flat = Flatten()(dropout2)
dense1 = Dense(64, activation='relu')(pool2_flat)
dropout3 = Dropout(dropout_rate_2)(dense1)
dense2 = Dense(32, activation='relu')(dropout3)
dropout4 = Dropout(dropout_rate_2)(dense2)
predictions = Dense(n_classes, activation='softmax')(dropout4)
model = Model(inputs=x, outputs=predictions)

model.summary()


In [ ]:
loss = sparse_categorical_crossentropy
optimizer = 'adam'
# Compilation
#############
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
history = model.fit(x=X_train, y=y_train,
                    validation_data=(X_test, y_test),
                    epochs=epoch_count,
                    batch_size=batch_size)

In [ ]:
model.save('kettlebell_model.h5')

In [ ]:
import socket
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(('192.168.4.1',8000))

In [ ]:
while True:
    data = s.recv(3200)
    rows = []
    for j in range(200):
        offset = j*8*2    
        b=data[offset:offset+16]
        time_ms = int.from_bytes(b[0:4], byteorder='big', signed=True)
        imus = [time_ms]
        for i in range(6):
          imus.append(int.from_bytes(b[i*2+4:i*2+6], byteorder='big', signed=True))
        rows.append(imus)
    df = pd.DataFrame(rows,columns=['time_ms','acc_x','acc_y','acc_z','gyr_x','gyr_y','gyr_z'])
    X = np.array([])
    target_dict={0:'DEADLIFT',1:'HALO',2:'CLEAN', 3:'SNATCH',4:'SWING',5:'NOTHING'}
    X=(df.loc[:,'acc_x':'gyr_z'].to_numpy()+32768)/65536
    X=X.reshape(1,200,6)
    pred = model.predict(X)
    print(target_dict[np.argmax(pred)], np.max(pred), end='           \r')


In [ ]:
max_amplitudo_csatorna_azonositasa()

In [ ]:
periodus_azonositas_vizualizácio()

In [ ]:
def colored(r, g, b, text):
    return "\033[38;2;{};{};{}m{} \033[38;2;255;255;255m".format(r, g, b, text)
  
text = 'Hello, World'
colored_text = colored(255, 0, 0, text)
print(colored_text)